In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df_questions_ymd = pd.read_csv('./data/silver/df_questions_by_period_YYYY-MM-DD.csv', index_col=0)
df_meetings_ymd = pd.read_csv('./data/silver/df_meetings_by_period_YYYY-MM-DD.csv', index_col=0)

df_meps = pd.read_csv('./data/silver/mep_membership_timeline.csv')

In [3]:
all_periods = sorted(set(df_questions_ymd.columns.tolist() + df_meetings_ymd.columns.tolist()))

# Fill missing periods with 0
df_questions = df_questions_ymd.reindex(all_periods, axis=1, fill_value=0)
df_meetings = df_meetings_ymd.reindex(all_periods, axis=1, fill_value=0)

# Combine dataframes



In [4]:
# I need to transform the columns into rows
df_questions_melted = df_questions.melt(ignore_index=False, var_name='date', value_name='questions')
df_meetings_melted = df_meetings.melt(ignore_index=False, var_name='date', value_name='meetings')

# Reset index to handle duplicate indices
df_questions_melted = df_questions_melted.reset_index()
df_meetings_melted = df_meetings_melted.reset_index()

# now join the two dataframes
df_combined = pd.merge(df_questions_melted, df_meetings_melted, 
                      on=['member_id', 'date'])

# Aggregate the data by week
# df_combined['date'] = pd.to_datetime(df_combined['date']).dt.strftime('%Y-%U')
# df_combined = df_combined.groupby(['member_id', 'date']).sum().reset_index()


# Add MEP data

In [42]:
df_combined["member_id"] = df_combined["member_id"].astype(int)

df = df_meps.merge(
    df_questions_melted,
    right_on=["member_id", "date"],
    left_on=["ID", "date"],
    how="left",
).merge(
    df_meetings_melted,
    right_on=["member_id", "date"],
    left_on=["ID", "date"],
    how="left",
)

# Add dummies

In [ ]:
new_columns = []

# Drop political group 0 as reference category
political_groups = df["POLITICAL_GROUP"].unique()
political_groups = political_groups[political_groups != 0]  # Remove 0
for group in political_groups:
    col_name = f"political_group_{int(group)}"
    new_columns.append(col_name)
    df[col_name] = (df["POLITICAL_GROUP"] == group).astype(int)



# DROP country FRA as reference
countries = df["COUNTRY"].unique()
countries = countries[countries != "FRA"]  #FRA as Reference
for country in countries:
    col_name = f"country_{country}"
    new_columns.append(col_name)
    df[col_name] = (df["COUNTRY"] == country).astype(int)

# Clean columns

In [ ]:
cols_to_ignore = [
    "ID",
    "member_id_x",
    "member_id_y",
    "POLITICAL_GROUP",
    "COUNTRY",
    " - CHAIR",
    " - CHAIR_VICE",
    " - MEMBER",
    " - MEMBER_PARLIAMENT",
    " - PRESIDENT",
    " - PRESIDENT_PARLIAMENT_STOA",
    " - PRESIDENT_VICE",
    " - QUAESTOR",
]

In [56]:
df.to_csv('./data/gold/panel_data_3.csv', index=False)

In [54]:
df[['ID', 'questions', 'meetings', *df_meps.columns[5:]]].groupby('ID').sum().sort_values('meetings')

,questions,meetings,EU_POLITICAL_GROUP - MEMBER,NATIONAL_CHAMBER - MEMBER,EU_INSTITUTION - MEMBER,DELEGATION_PARLIAMENTARY_ASSEMBLY - MEMBER_SUBSTITUTE,COMMITTEE_PARLIAMENTARY_STANDING - MEMBER_SUBSTITUTE,DELEGATION_JOINT_COMMITTEE - MEMBER,COMMITTEE_PARLIAMENTARY_STANDING - MEMBER,WORKING_GROUP - MEMBER,...,- PRESIDENT_PARLIAMENT_STOA,NATIONAL_CHAMBER - TREASURER_CO,WORKING_GROUP -,EU_POLITICAL_GROUP - CHAIR_BUREAU,EU_POLITICAL_GROUP -,- QUAESTOR,EU_INSTITUTION - PRESIDENT,- PRESIDENT,- PRESIDENT_VICE,EU_POLITICAL_GROUP - PRESIDENT_VICE
ID,,,,,,,,,,,,,,,,,,,,,
129407,82.0,0.0,1151.0,1151.0,1151.0,358.0,1151.0,0.0,1149.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
124966,78.0,0.0,1250.0,1250.0,1250.0,0.0,0.0,0.0,1250.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
124967,14.0,0.0,1250.0,1250.0,1250.0,0.0,1250.0,0.0,1250.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
124968,85.0,0.0,722.0,722.0,722.0,0.0,722.0,0.0,722.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
124969,30.0,0.0,711.0,711.0,711.0,0.0,707.0,63.0,711.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197802,58.0,528.0,1923.0,1923.0,1923.0,129.0,1920.0,0.0,1920.0,1588.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28298,100.0,616.0,1256.0,3179.0,3179.0,0.0,1439.0,6.0,2353.0,1506.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96932,87.0,685.0,3173.0,3173.0,3173.0,0.0,3166.0,0.0,3170.0,1555.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
df.sort_values(by=['member_id', 'date']).to_csv('./data/gold/panel_data.csv')


PermissionError: [Errno 13] Permission denied: './data/gold/panel_data.csv'

In [ ]:

df_to_plot = df_combined[df_combined['member_id'] == 1927.0]

plt.figure(figsize=(10, 6))
plt.plot(df_to_plot['date'], df_to_plot['meetings'], label='Meetings')
plt.plot(df_to_plot['date'], df_to_plot['questions'], label='Questions')
plt.legend()
plt.show()

